In [1]:
import sys, os
project_root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

if project_root_path not in sys.path:
    sys.path.append(project_root_path)
print(project_root_path)

import datetime as dt
import numpy as np
import pandas as pd
import plotly.express as px

import qpt_stress_test.services.positions as positions
import qpt_stress_test.core.config as config
import qpt_stress_test.db.tasks as db_tasks


/home/skingham/Projects/galaxy_sk/qpt_stress_test


In [2]:
# Services
trade_service = positions.TradeService()


In [3]:
# Test db connection
pg_conn = db_tasks.pgsql_engine()
reporting_dt = dt.date.today() - dt.timedelta(days=1)
sql = (
        "select "
        "   symbol_exch as instrument, "
        "   upper(exchange) as exchange, "
        "   instrument_type, "
        "   expiration_time, "
        "   is_linear,  "
        "   contract_size,  "
        "   qty_multiplier,  "
        "   case when is_linear > 0 then currency_position else currency_settlement end as underlying "
        "from "
        "   Trading.definition.instrument_reference "
        "where "
        "   symbol_bfc is not null and expiration_time >= '{}' "
        "limit 5000").format(reporting_dt.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
df = pd.read_sql(sql, con=pg_conn)

display(df)

,instrument,exchange,instrument_type,expiration_time,is_linear,contract_size,qty_multiplier,underlying
0,ANKR-USD,HUOBI,SWAP,2099-12-31 23:00:00,0.0,10.0000,1,ANKR
1,XEMBTC,HUOBI,SPOT,2099-12-31 08:00:00,NaN,0.0001,1000000000,None
2,RVNBTC,HUOBI,SPOT,2099-12-31 08:00:00,NaN,0.0001,1000000000,None
3,AVAX-USD,HUOBI,SWAP,2099-12-31 08:00:00,0.0,10.0000,1,AVAX
4,DOGEBTC,HUOBI,SPOT,2099-12-31 08:00:00,NaN,0.0001,1000000000,None
...,...,...,...,...,...,...,...,...
4995,BABA-1230,FTX,FUTURE,2022-12-30 21:00:00,1.0,NaN,1000000000,BABA
4996,BNB-1230,FTX,FUTURE,2022-12-30 03:00:00,1.0,NaN,1000000000,BNB
4997,BILI-1230,FTX,FUTURE,2022-12-30 21:00:00,1.0,NaN,1000000000,BILI
4998,BSV-1230,FTX,FUTURE,2022-12-30 03:00:00,1.0,NaN,1000000000,BSV


### Get all symbols and for the period from the beginning of the historic period

In [3]:
reporting_dt = dt.date.today() - dt.timedelta(days=1)
hist_start_dt = reporting_dt - dt.timedelta(days=365)
report_utc_dtt = dt.datetime(year=2022, month=10, day=7, hour=22, minute=0, second=0)



In [4]:
cme_positions = trade_service.cme_positions(report_utc_dtt)
cme_positions_columns, cme_positions_data = cme_positions.as_list()
cme_positions_map = cme_positions.as_map()
cme_positions_df = cme_positions.as_dataframe()
display(cme_positions_df)

/home/skingham/Projects/galaxy_sk/qpt_stress_test/qpt_stress_test/db/repositories/ms_trading.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(self._query, con=self._db_conn)


,instrument,exchange,account,position,notional,price,unrealized_pnl,instrument_type,expiration_time,is_linear,underlying,EntryCost
0,BTCV2,ED&F,UNMBF222,427,41440350.0,19410.0,2774875.0,CME FUTURE,2022-10-29,1,BTC,38665475.0
1,BTCX2,ED&F,UNMBF222,37,3592700.0,19420.0,240925.0,CME FUTURE,2022-11-26,1,BTC,3351775.0
2,ETHV2,ED&F,UNMBF222,382,25297950.0,1324.5,1563345.0,CME FUTURE,2022-10-29,1,ETH,23734605.0
3,ETHX2,ED&F,UNMBF222,12,794100.0,1323.5,-100.0,CME FUTURE,2022-11-26,1,ETH,794200.0


In [7]:
contracts = trade_service.active_contracts(hist_start_dt, reporting_dt)
contracts_columns, contracts_data = contracts.as_list()
contracts_map = contracts.as_map()
contracts_df = contracts.as_dataframe()

display(contracts_df)

,instrument,exchange,symbol_exch,endpoint,instrument_type,expiration_time,is_linear,contract_size,qty_multiplier,underlying
0,H_P_ANKR-USD-SWAP,HUOBI,ANKR-USD,HUBI,SWAP,2099-12-31 23:00:00,0.0,10.0000,1,ANKR
1,XEMBTC,HUOBI,XEMBTC,HUBI,SPOT,2099-12-31 08:00:00,NaN,0.0001,1000000000,None
2,RVNBTC,HUOBI,RVNBTC,HUBI,SPOT,2099-12-31 08:00:00,NaN,0.0001,1000000000,None
3,H_P_AVAX-USD-SWAP,HUOBI,AVAX-USD,HUBI,SWAP,2099-12-31 08:00:00,0.0,10.0000,1,AVAX
4,DOGEBTC,HUOBI,DOGEBTC,HUBI,SPOT,2099-12-31 08:00:00,NaN,0.0001,1000000000,None
...,...,...,...,...,...,...,...,...,...,...
9995,D_I_ETH_USD_220614_1700_P,DERIBIT,ETH-14JUN22-1700-P,DERI,PUT,2022-06-14 08:00:00,0.0,1.0000,1000000000,ETH
9996,D_I_ETH_USD_220614_1400_C,DERIBIT,ETH-14JUN22-1400-C,DERI,CALL,2022-06-14 08:00:00,0.0,1.0000,1000000000,ETH
9997,D_I_BTC_USD_220614_31000_P,DERIBIT,BTC-14JUN22-31000-P,DERI,PUT,2022-06-14 08:00:00,0.0,1.0000,1000000000,BTC
9998,D_I_ETH_USD_220614_1350_P,DERIBIT,ETH-14JUN22-1350-P,DERI,PUT,2022-06-14 08:00:00,0.0,1.0000,1000000000,ETH
